In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

import carrega_imagens

## 1. Carregando as entradas (X) e saídas (y)

Criamos um script específico (*carrega_imagens.py*) para carregar e transformar as imagens em tensores, e que já faz a separação das entradas e saídas em treino (80%) e teste (20%):

In [6]:
X_train, X_test, y_train, y_test = carrega_imagens.load()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Aplicando *One Hot Encoding* na saída, para conseguirmos realizar a classificação, e ajustando as dimensões das entradas:

In [7]:
y_train_OH = to_categorical(y_train)
y_test_OH = to_categorical(y_test)
y_val_OH = to_categorical(y_val)
X_train_2D = X_train.reshape(3200, 400*400)
X_test_2D = X_test.reshape(1000, 400*400)
X_val_2D = X_val.reshape(800, 400*400)

## 2. Comparando a CNN com um XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold

xgbc = XGBClassifier(learning_rate=0.1, objective='multi:softmax', num_class=10,
                    evals=(X_val_2D, y_val), verbosity=2, early_stopping_rounds=2, eval_metric='accuracy',
                    random_state=42)
print(xgbc)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=2,
              enable_categorical=False, eval_metric='accuracy',
              evals=(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
                     array([7., 1.,...
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_class=10, num_parallel_tree=None, objective='multi:softmax',
              predictor=

In [17]:
xgbc_history = xgbc.fit(X_train_2D, y_train)

XGBoostError: bad allocation

In [ ]:
# - cross validataion
scores = cross_val_score(xgbc, X_train_2D, y_train_OH, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())
 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_train_2D, y_train_OH, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

#ypred = xgbc.predict(X_test)
#cm = confusion_matrix(y_test_OH, ypred)
#print(cm)